In [67]:
from contextlib import contextmanager
import sqlite3
import yaml

In [68]:
! pwd

/Users/jeremy/projects/hyperborea3/scripts/spell_desc


In [69]:
load_folder = "lvl1"

In [70]:
@contextmanager
def db_cur():
    URI = f"../../hyperborea3/hyperborea.sqlite3"
    con = sqlite3.connect(URI, check_same_thread=False)
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    yield cur
    con.commit()
    con.close()

Test connection to db

In [71]:
with db_cur() as cur:
    cur.execute("""
        SELECT *
          FROM spells;
    """)
    result = [dict(x) for x in cur.fetchall()]
result[:1]

[{'spell_id': 1,
  'spell_name': 'Acid Arrow',
  'rng': '30 feet',
  'dur': 'special',
  'reversible': 0,
  'pp': 175,
  'spell_desc': '<p>A magic arrow darts from the finger of the caster. On a successful attack roll (<i>dexterity</i> modifier applies), the acid arrow strikes for 1d4+1 hp physical damage, plus an additional 2d4 hp acid damage in the same round. Magicians (but not other sorcerers) enjoy a +1 bonus to the attack roll for every 2 CA levels (CA 3–4 = +2, CA 5–6 = +3, and so forth). Acid damage will persist for higher level sorcerers:</p> <ul> <li>1 extra round for CA 4–6</li> <li>2 extra rounds for CA 7–9</li> <li>3 extra rounds for CA 10 or greater.</li> </ul> <p>For example, an <i>acid arrow</i> fired by a CA 12 sorcerer on round 1 would inflict 1d4+1 hp base damage plus 2d4 hp acid damage on round 1, 2d4 hp acid damage on round 2, 2d4 hp acid damage on round 3, and a final 2d4 hp acid damage on round 4. The acid may ruin armour or clothing per referee discretion. <i>N.

In [72]:
files = ! ls {load_folder}/*
files

['lvl1/Alarm.yml',
 'lvl1/Alter_Self.yml',
 'lvl1/Animate_Carrion.yml',
 'lvl1/Auditory_Glamour.yml',
 'lvl1/Befriend_Animals.yml',
 'lvl1/Black_Hand.yml',
 'lvl1/Bless.yml',
 'lvl1/Bless_Oil_or_Water.yml',
 'lvl1/Burning_Hands.yml',
 'lvl1/Ceremony_of_Consecration.yml',
 'lvl1/Charm_Person.yml']

In [73]:
def load_yml_spell(file_name):
    with open(file_name, "r") as f:
        spell = yaml.safe_load(f)
    return spell

In [74]:
def spell_preload_check(spell):
    with db_cur() as cur:
        cur.execute(
            """
            SELECT *
            FROM spells
            WHERE spell_id = ?;
            """,
            (spell["id"],),
        )
        db_spell = cur.fetchone()
    assert spell["name"] == db_spell["spell_name"]
    assert isinstance(spell["id"], int)
    assert isinstance(spell["pp"], int)
    assert isinstance(spell["Rng"], (str, int))
    assert isinstance(spell["Dur"], str)
    assert spell["Rev"] in [0, 1]
    assert isinstance(spell["Desc"], str)



In [75]:
def update_spell(spell):
    with db_cur() as cur:
        cur.execute(
            """
            UPDATE spells
            SET rng = ?
                , dur = ?
                , reversible = ?
                , pp = ?
                , spell_desc = ?
            WHERE spell_id = ?;
            """,
            (
                spell["Rng"],
                spell["Dur"],
                spell["Rev"],
                spell["pp"],
                spell["Desc"],
                spell["id"],
            ),
        )
    return 

### Test parsing all the files

In [76]:
for f in files:
    spell = load_yml_spell(f)
    spell_preload_check(spell)

Do it for real

In [77]:
for f in files:
    spell = load_yml_spell(f)
    spell_preload_check(spell)
    update_spell(spell)

In [78]:
load_yml_spell(files[-1])

{'name': 'Charm Person',
 'id': 53,
 'pp': 183,
 'Rng': '120 feet',
 'Dur': 'special',
 'Rev': 0,
 'Desc': '<p>The target of this spell (human, humanoid, or giant) must make a <i>sorcery</i> saving throw, modified by <i>willpower adjustment</i>, if applicable. If the save fails, the victim views the caster as a close and trusted comrade to be protected and defended for the duration of the spell. If the ensorcelled victim understands the caster’s language, the caster can “suggest” the victim perform various tasks, so long as they are not diametrically opposite to the victim’s nature.</p><p>The spell <i>dispel magic</i> can cancel this spell. Otherwise, additional <i>sorcery</i> saving throws (<i>willpower adjustment</i> applies) are allowed to break the spell at a time predicated on the targets intelligence (IN) score.:<ul><li>13–18 IN = new save every 6 turns</li><li>9–12 IN = new save every 24 hours</li><li>3–8 IN = new save once per week</li></ul></p>'}